In [128]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import itertools as it
import statsmodels.api
import statsmodels.formula.api as smf
import sklearn.linear_model as lm

from sklearn.metrics import mean_squared_error, r2_score

data = pd.read_csv("AFAPy.csv")
print(data.columns)
datat = np.transpose(data)
names = data.columns[7:]
for col in data.columns:
    if col != 'DropLoc':
        pd.to_numeric(data[col])

Index(['Gamenumber', 'Placement', 'RawTTK', 'AdjustedTTK', 'Dub', 'MSRK',
       'DropLoc', 'Tom', 'Jackson', 'Ishjot', 'David', 'Ruvan', 'Austin',
       'Theo', 'Josh', 'Vinny', 'Andrew', 'Oliver', 'Michael', 'Felix'],
      dtype='object')


In [129]:
form = "Dub ~ "
for i in range(0,len(names)):
    form +=""+names[i]+""
    if i < len(names) - 1:
        form += " + "
        
logit = statsmodels.discrete.discrete_model.MNLogit.from_formula(form, data).fit(method='bfgs', maxiter=1000)
print(logit.summary())

Optimization terminated successfully.
         Current function value: 0.480852
         Iterations: 114
         Function evaluations: 115
         Gradient evaluations: 115
                          MNLogit Regression Results                          
Dep. Variable:                    Dub   No. Observations:                  791
Model:                        MNLogit   Df Residuals:                      777
Method:                           MLE   Df Model:                           13
Date:                Tue, 24 Jul 2018   Pseudo R-squ.:                 0.05156
Time:                        00:01:26   Log-Likelihood:                -380.35
converged:                       True   LL-Null:                       -401.03
                                        LLR p-value:                 8.356e-05
     Dub=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.6714      1.275   

In [130]:
form = "AdjustedTTK ~ "
for i in range(0,len(names)):
    form +=""+names[i]+""
    if i < len(names) - 1:
        form += " + "
        
TTKReg = statsmodels.genmod.generalized_linear_model.GLM.from_formula(form, data).fit()
print(TTKReg.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:            AdjustedTTK   No. Observations:                  791
Model:                            GLM   Df Residuals:                      777
Model Family:                Gaussian   Df Model:                           13
Link Function:               identity   Scale:                   39.0922968952
Method:                          IRLS   Log-Likelihood:                -2565.2
Date:                Tue, 24 Jul 2018   Deviance:                       30375.
Time:                        00:01:27   Pearson chi2:                 3.04e+04
No. Iterations:                     2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.2387      2.330      3.536      0.000       3.672      12.806
Tom            1.9964      0.822      2.429      0.0

In [132]:
form = "Placement ~ "
for i in range(0,len(names)):
    form +=""+names[i]+""
    if i < len(names) - 1:
        form += " + "
        
APReg = statsmodels.genmod.generalized_linear_model.GLM.from_formula(form, data).fit()
print(APReg.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Placement   No. Observations:                  791
Model:                            GLM   Df Residuals:                      777
Model Family:                Gaussian   Df Model:                           13
Link Function:               identity   Scale:                   71.9163678554
Method:                          IRLS   Log-Likelihood:                -2806.3
Date:                Tue, 24 Jul 2018   Deviance:                       55879.
Time:                        00:01:35   Pearson chi2:                 5.59e+04
No. Iterations:                     2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     24.8432      3.160      7.861      0.000      18.649      31.038
Tom           -5.7017      1.115     -5.115      0.0

In [15]:
print("Name \t\tMultiplicative Effect on Dub Odds Ratio \t Effect on teams AdjTTK \t Effect on AP")
for i in range(0, len(names)):
    print(names[i], "\t\t", round(np.exp(logitCoefs[0][i]),3),"\t \t \t\t\t\t",round(TTKCoefs[i],3),"\t\t\t\t",round(APCoefs[i],3))
print("Coefficents \t",round(np.exp(logitInt[0]),3),"\t\t\t\t\t\t",round(TTKInt,3),"\t\t\t\t",round(APInt,3))

Name 		Multiplicative Effect on Dub Odds Ratio 	 Effect on teams AdjTTK 	 Effect on AP
Tom 		 1.851 	 	 				 1.856 				 -5.036
Jackson 		 1.498 	 	 				 0.065 				 -2.711
Ishjot 		 1.493 	 	 				 -0.439 				 -3.602
David 		 1.193 	 	 				 -0.99 				 -2.705
Ruvan 		 1.41 	 	 				 -1.568 				 -2.35
Austin 		 1.446 	 	 				 -0.785 				 -2.114
Theo 		 1.54 	 	 				 0.727 				 -3.814
Josh 		 1.516 	 	 				 1.586 				 -3.759
Vinny* 		 1.998 	 	 				 1.344 				 -4.778
Andrew 		 0.999 	 	 				 -2.135 				 -0.634
Oliver 		 1.248 	 	 				 -1.096 				 -3.119
Michael 		 1.99 	 	 				 6.261 				 -12.242
Coefficents 	 0.098 						 8.17 				 22.839


In [16]:
combos = set(it.combinations(names, 4))
odds = []
probs = []
teamOdds = {}
teamProbs = {}
for team in combos:
    odd = logitInt[0]
    for player in team:
        odd = odd + logitCoefs[0][names.index(player)]
    odds.append(odd)
    probs.append(np.exp(odd)/(1+np.exp(odd)))
    teamOdds[team] = odd
    teamProbs[team] = np.exp(odd)/(1+np.exp(odd))
teamProbs
sorted(teamProbs.items(), key=lambda x: x[1])

[(('David', 'Ruvan', 'Andrew', 'Oliver'), 0.17066425912452229),
 (('David', 'Austin', 'Andrew', 'Oliver'), 0.17419678589009499),
 (('Ishjot', 'David', 'Andrew', 'Oliver'), 0.17887591314222334),
 (('Jackson', 'David', 'Andrew', 'Oliver'), 0.17933746807395515),
 (('David', 'Josh', 'Andrew', 'Oliver'), 0.18108309932203784),
 (('David', 'Theo', 'Andrew', 'Oliver'), 0.18342514744685773),
 (('David', 'Ruvan', 'Austin', 'Andrew'), 0.19250657441995739),
 (('Ishjot', 'David', 'Ruvan', 'Andrew'), 0.19755986341421514),
 (('Jackson', 'David', 'Ruvan', 'Andrew'), 0.19805800018680966),
 (('Ruvan', 'Austin', 'Andrew', 'Oliver'), 0.199561518347248),
 (('David', 'Ruvan', 'Josh', 'Andrew'), 0.19994145756026602),
 (('Ishjot', 'David', 'Austin', 'Andrew'), 0.20151382486636593),
 (('Jackson', 'David', 'Austin', 'Andrew'), 0.20201942140729801),
 (('David', 'Ruvan', 'Theo', 'Andrew'), 0.20246710863451431),
 (('David', 'Austin', 'Josh', 'Andrew'), 0.20393097065168048),
 (('Ishjot', 'Ruvan', 'Andrew', 'Oliver'

In [17]:
combos = set(it.combinations(names, 4))
TTKs = []
teamTTKs = {}
for team in combos:
    kills = TTKInt
    for player in team:
        kills = kills + TTKCoefs[names.index(player)]
    TTKs.append(kills)
    teamTTKs[team] = kills
sorted(teamTTKs.items(), key=lambda x: x[1])

[(('David', 'Ruvan', 'Andrew', 'Oliver'), 2.3820538216346385),
 (('Ruvan', 'Austin', 'Andrew', 'Oliver'), 2.5863187552905096),
 (('David', 'Ruvan', 'Austin', 'Andrew'), 2.6924187571165561),
 (('Ishjot', 'Ruvan', 'Andrew', 'Oliver'), 2.9323614913057767),
 (('Ishjot', 'David', 'Ruvan', 'Andrew'), 3.0384614931318232),
 (('David', 'Austin', 'Andrew', 'Oliver'), 3.1646432672333162),
 (('Ishjot', 'Ruvan', 'Austin', 'Andrew'), 3.2427264267876943),
 (('Jackson', 'Ruvan', 'Andrew', 'Oliver'), 3.4367220019540881),
 (('Ishjot', 'David', 'Andrew', 'Oliver'), 3.5106860032485834),
 (('Jackson', 'David', 'Ruvan', 'Andrew'), 3.5428220037801346),
 (('Ishjot', 'Austin', 'Andrew', 'Oliver'), 3.7149509369044544),
 (('David', 'Ruvan', 'Austin', 'Oliver'), 3.731391473068145),
 (('Jackson', 'Ruvan', 'Austin', 'Andrew'), 3.7470869374360056),
 (('Ishjot', 'David', 'Austin', 'Andrew'), 3.8210509387305009),
 (('Jackson', 'David', 'Andrew', 'Oliver'), 4.0150465138968947),
 (('Ishjot', 'David', 'Ruvan', 'Oliver'),

In [18]:
combos = set(it.combinations(names, 4))
APs = []
TeamAPs = {}
for team in combos:
    Place = APInt
    for player in team:
        Place = Place + APCoefs[names.index(player)]
    APs.append(Place)
    TeamAPs[team] = Place
sorted(TeamAPs.items(), key=lambda x: x[1])

[(('Tom', 'Theo', 'Vinny*', 'Michael'), -3.0316297802402925),
 (('Tom', 'Josh', 'Vinny*', 'Michael'), -2.9765513537496489),
 (('Tom', 'Ishjot', 'Vinny*', 'Michael'), -2.8196641330988754),
 (('Tom', 'Vinny*', 'Oliver', 'Michael'), -2.3364498132508746),
 (('Tom', 'Theo', 'Josh', 'Michael'), -2.0123220288579589),
 (('Tom', 'Jackson', 'Vinny*', 'Michael'), -1.928575324623262),
 (('Tom', 'David', 'Vinny*', 'Michael'), -1.9227347695891748),
 (('Tom', 'Ishjot', 'Theo', 'Michael'), -1.8554348082071854),
 (('Tom', 'Ishjot', 'Josh', 'Michael'), -1.8003563817165418),
 (('Theo', 'Josh', 'Vinny*', 'Michael'), -1.7539966109748342),
 (('Ishjot', 'Theo', 'Vinny*', 'Michael'), -1.597109390324059),
 (('Tom', 'Ruvan', 'Vinny*', 'Michael'), -1.568081782322734),
 (('Ishjot', 'Josh', 'Vinny*', 'Michael'), -1.5420309638334153),
 (('Tom', 'Theo', 'Oliver', 'Michael'), -1.3722204883591846),
 (('Tom', 'Austin', 'Vinny*', 'Michael'), -1.3314497534259448),
 (('Tom', 'Josh', 'Oliver', 'Michael'), -1.31714206186853